# <b> CAPSTONE : Where is the best neighborhood in Denpasar City, Bali to open New Fast Food Restaurant ?</b>

by : Arya Anandita

### <b> INTRODUCTION :  </b>

In this Assignment, I will analyze a city in island of god, Bali Island neighborhood, which is Denpasar City neighborhood.in this assigment I will analyze where is the best neighborhood to open New Fast Food Restaurant in Denpasar City

### <b> PROBLEM :  </b>

The aim of this Assigment is to try to help business owners to open new fast food restaurant in the neighborhood depending on the experiences that the neighborhood has.the requirements of the best Neighborhood to open new fast food is :
<li>there is no fast food restaurant in or near that neighborhood</li>
<li>restaurant needs to be strategically located near public or crowded place in Denpasar area</li>
<li>Confirm any assumption by means of modeling and testing the data.</li>

### <b> DATA REQUIREMENT :  </b>

##### <b>1. The list of Denpasar city neighborhood (Kecamatan) </b>
To get the neighborhood (Kecamatan) in Denpasar, we scrap the data from wikipedia link :
https://id.wikipedia.org/wiki/Daftar_kecamatan_dan_kelurahan_di_Kota_Denpasar
In this Wikipedia page, there is several information about neighborhood in Denpasar, but for this assignment we just need information about the name of all neighborhood in Denpasar

##### <b>2. Longitude and latitude information of each Neighborhood </b>
The data scraping from the Wikipedia page does not give information about the coordinates for each neighborhood. So we can use Nominatim OpenStreetMap API in order to get *latitude* and *longitude* for each neighborhood.
Using Nominatim OpenStreetMap API in python we can use geopy library and import geopy.geocoders.Nominatim package into a notebook.
Using nominatim we can pass neighborhood keywords into nominatim objects and get the representing latitude and longitude so we can add this information into the neighborhood table for Denpasar

##### <b>3. Total Population in each Neighborhood </b>
to know which neighborhood has a high or dense population, we need to download population data of Denpasar city in this link : 
https://denpasarkota.bps.go.id/indicator/12/49/1/proyeksi-penduduk-kota-denpasar.html
this is the website of Denpasar City's central bureau of statistic that contain population data for each neighborhood in Denpasar city

##### <b>4. List of all venues and places from all neighborhood in Denpasar  </b>
to know all venues & places from all neighborhood in Denpasar city, we use Foursquare API .With Foursquare API we can make a call containing neighborhood information so we can gain information about the places or venues

Combine all data that we gather into dataset to demonstrate information :
<li>how populated each neighborhoods is</li>
<li>which neighborhood still don't have fast food restaurant in its area </li>
<li>which neighborhood have many public venues </li>
<li>which neighborhood should targeted to open new fast food restaurant </li>

### <b> METHODOLOGY :  </b>

In this part of the section, I will collecting data (data scrapping) from wikipedia page in order to get neighborhood information for Denpasar City,and then I will collect population data in each neighborhood & combine that data with neighborhood information data from wikipedia.  After getting that informations, I will use name of the neighborhood as a keyword to providing information about neighborhood coordinates (latitude and longitude) using Nominatim with geopy.geocoders.Nominatim package. Using coordinates for each neighborhood i will use Foursquare API to get relevant venues and places near the given latitude and longitude. Using that information we create a pandas dataframe to analyze which neighborhood fit the requirements of the best Neighborhood to open new fast food.


#### <b> Neighborhood Information and Population </b>

In [73]:
#Import the libraries
import pandas as pd
import requests
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup

In [74]:
#DATA COLLECTION
#Denpasar Neighborhood
denpasar_url = 'https://id.wikipedia.org/wiki/Daftar_kecamatan_dan_kelurahan_di_Kota_Denpasar'
denpasar_wiki = requests.get(denpasar_url)
soup = BeautifulSoup(denpasar_wiki.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
df
df_den = pd.DataFrame(df[0])
df=df_den[['Kecamatan']]
df = df.drop_duplicates(keep = 'first')
df= df[(df.Kecamatan != 'TOTAL')]
df.rename(columns={'Kecamatan':'Neighborhood'}, inplace=True)
df


,Neighborhood
0,Denpasar Barat
2,Denpasar Selatan
4,Denpasar Timur
6,Denpasar Utara


We got information that desribe Denpasar City have 4 Neighborhood

In [80]:
#Population Data
df_pop = pd.read_csv('https://raw.githubusercontent.com/DonCorleone880/Coursera_Capstone/main/Denpasar_Neighborhood_Population1.csv')
df_pop.rename(columns={'KECAMATAN':'Neighborhood','JUMLAH PENDUDUK 2020':"Population in 2020"}, inplace=True)
df_pop


,Neighborhood,Population in 2020
0,Denpasar Selatan,311590
1,Denpasar Timur,162220
2,Denpasar Barat,278020
3,Denpasar Utara,211070


as we can see in the dataset above we have population information for  each 4 neighborhood in Denpasar city

#### <b> Neighborhood Latitude & longitude </b>

In [81]:
# Create Nominatim object as 'geolocator'
geolocator = Nominatim(user_agent='explorer')

In [85]:
# All of these function will provide information about latitude and longitude for neighborhood
def get_latitude_denpasar(neighborhood):
    location = geolocator.geocode(f'{neighborhood}, Denpasar, Indonesia')
    latitude = location.latitude
    return latitude

def get_longitude_denpasar(neighborhood):
    location = geolocator.geocode(f'{neighborhood}, Denpasar, Indonesia')
    longitude = location.longitude
    return longitude

In [87]:
#combine neighborhood df_pop dataset with longitude latitude data
df_pop['Latitude'] = df_pop['Neighborhood'].apply(get_latitude_denpasar)
df_pop['Longitude'] = df_pop['Neighborhood'].apply(get_longitude_denpasar)
df_pop

,Neighborhood,Population in 2020,Latitude,Longitude
0,Denpasar Selatan,311590,-8.706134,115.225676
1,Denpasar Timur,162220,-8.632914,115.246211
2,Denpasar Barat,278020,-8.662244,115.198067
3,Denpasar Utara,211070,-8.624513,115.209009


#### <b> Map Visualization </b>
Visualizing map using Folium API with OpenStreetMap view with information of neighborhood from dataframes



In [88]:
#get coordinate of Denpasar
address = 'Denpasar'

location = geolocator.geocode(address)
denpasar_latitude = location.latitude
denpasar_longitude = location.longitude
print(f'Coordinates of Denpasar are {denpasar_latitude}, {denpasar_longitude}')

Coordinates of Denpasar are -8.6524973, 115.2191175


In [93]:
#Folium OpenStreetMap of Jakarta Neighborhood
denpasar_map = folium.Map(location=[denpasar_latitude, denpasar_longitude], zoom_start=11)

for latitude, longitude, neighborhood in zip(df_pop['Latitude'], df_pop['Longitude'], df_pop['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='red',
        fill=True
        ).add_to(denpasar_map)
    
denpasar_map

#### <b> Get Nearby Venues </b>

Defining Foursquare API Credentials and Version

In [94]:
CLIENT_ID = '0RCP3HE3LQTUJYAOZWSHJNWNDCE4QRKJDJOW4LAWNLPN4CQF' # your Foursquare ID
CLIENT_SECRET = 'STJD4NFEBB1LBXU4MXJANBZOKPHAOTDCNRIAIGHIM2Y53ZVC' # your Foursquare Secret
VERSION = '20201228' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0RCP3HE3LQTUJYAOZWSHJNWNDCE4QRKJDJOW4LAWNLPN4CQF
CLIENT_SECRET:STJD4NFEBB1LBXU4MXJANBZOKPHAOTDCNRIAIGHIM2Y53ZVC


Create a Function that retrieves information about venues and places in given latitudes and longitudes Using Foursquare API

In [170]:
# Function that return latitude, longitude, venues, and venue_categories in neighborhood_df
def get_nearby_venues(names, latitudes, longitudes, radius=500):
    
    # create an empty list
    venues_list=[]
    
    # for loop that iterate through dataframe
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius
            )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

        
    # Create pandas dataframe from venues_list
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

denpasar_venues = get_nearby_venues(df_pop['Neighborhood'], df_pop['Latitude'], df_pop['Longitude'])

Denpasar Selatan
Denpasar Timur
Denpasar Barat
Denpasar Utara


In [172]:
print(f'Shape of denpasar_venues dataframe : {denpasar_venues.shape}\n')
print('denpasar_venues dataframe : ')
denpasar_venues

Shape of denpasar_venues dataframe : (23, 5)

denpasar_venues dataframe : 


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category
0,Denpasar Selatan,-8.706134,115.225676,DEF Studio Music,Recording Studio
1,Denpasar Selatan,-8.706134,115.225676,Depot Laris Sate Plecing Singaraja,Indonesian Restaurant
2,Denpasar Selatan,-8.706134,115.225676,Texas Fried Chicken,American Restaurant
3,Denpasar Selatan,-8.706134,115.225676,Palapa Fitness Centre,Gym / Fitness Center
4,Denpasar Timur,-8.632914,115.246211,Roti Canai & Teh Tarik Bunana,Indian Restaurant
5,Denpasar Timur,-8.632914,115.246211,CV. Pelangi (rex's) pastry & bakery ingredients,Bakery
6,Denpasar Barat,-8.662244,115.198067,Conato Bakery,Bakery
7,Denpasar Barat,-8.662244,115.198067,SD Muhammadiyah 3 Denpasar,Arcade
8,Denpasar Barat,-8.662244,115.198067,Terang bulan dan martabak 'Sedap Mantap',Food Truck
9,Denpasar Barat,-8.662244,115.198067,Warung Banyuwangi,Asian Restaurant


in this Denpasar venues list,we found out that, just <b>Denpasar Barat</b> that have fast food restaurant in its area

In [112]:
denpasar_venues.to_csv('denpasar_venues.csv')

In [113]:
denpasar_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category
Neighborhood,,,,
Denpasar Barat,14,14,14,14
Denpasar Selatan,4,4,4,4
Denpasar Timur,2,2,2,2
Denpasar Utara,3,3,3,3


In [114]:
print(f'There are {len(denpasar_venues["Venue Category"].unique())} uniques categories in Denpasar.')

There are 17 uniques categories in Denpasar.


In [108]:
# one hot encoding
denpasar_onehot = pd.get_dummies(denpasar_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
denpasar_onehot['Neighborhood'] = denpasar_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [denpasar_onehot.columns[-1]] + list(denpasar_onehot.columns[:-1])
denpasar_onehot = denpasar_onehot[fixed_columns]

denpasar_onehot.head()



,Neighborhood,American Restaurant,Arcade,Asian Restaurant,Bakery,Convenience Store,Department Store,Fast Food Restaurant,Food Court,Food Truck,Gym / Fitness Center,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Kitchen Supply Store,Noodle House,Print Shop,Recording Studio
0,Denpasar Selatan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,Denpasar Selatan,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,Denpasar Selatan,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Denpasar Selatan,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,Denpasar Timur,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [115]:
denpasar_grouped = denpasar_onehot.groupby('Neighborhood').mean().reset_index()
denpasar_grouped.head()

,Neighborhood,American Restaurant,Arcade,Asian Restaurant,Bakery,Convenience Store,Department Store,Fast Food Restaurant,Food Court,Food Truck,Gym / Fitness Center,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Kitchen Supply Store,Noodle House,Print Shop,Recording Studio
0,Denpasar Barat,0.000000,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.214286,0.071429,0.071429,0.0,0.00,0.071429,0.000000,0.071429,0.00
1,Denpasar Selatan,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.0,0.25,0.000000,0.000000,0.000000,0.25
2,Denpasar Timur,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.00,0.000000,0.000000,0.000000,0.00
3,Denpasar Utara,0.333333,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.333333,0.000000,0.00


In [116]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### <b>top 5 most common venue</b>

In [138]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
denpasar_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
denpasar_neighborhoods_venues_sorted['Neighborhood'] = denpasar_grouped['Neighborhood']

for ind in np.arange(denpasar_grouped.shape[0]):
    denpasar_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(denpasar_grouped.iloc[ind, :], num_top_venues)

denpasar_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Denpasar Barat,Food Truck,Convenience Store,Arcade,Kitchen Supply Store,Asian Restaurant
1,Denpasar Selatan,Recording Studio,Indonesian Restaurant,Gym / Fitness Center,American Restaurant,Kitchen Supply Store
2,Denpasar Timur,Bakery,Indian Restaurant,Recording Studio,Food Court,Arcade
3,Denpasar Utara,American Restaurant,Noodle House,Convenience Store,Food Court,Arcade


#### <b>Modelling</b>
After we get data about top 5 most common venue for each neighborhood in Denpasar we can begin create a clustering model using K-Means Clustering library from Scikit-Learn
We will run the K-Means Clustering to cluster and segment the neighborhood into 2 different clusters based on type of venues and places.

In [139]:
# set number of clusters
kclusters = 2

# instantiate kmeans model
kmeans = KMeans(n_clusters=kclusters, random_state=0)

denpasar_cluster = denpasar_grouped.drop(columns=['Neighborhood'])

Clustering in Denpasar Neighborhood

In [140]:
# fit the data
denpasar_kmeans = kmeans.fit(denpasar_cluster)

# check cluster labels generated for each row in the dataframe
denpasar_kmeans.labels_

array([0, 0, 1, 0])

In [141]:

# add clustering labels
denpasar_neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

denpasar_merged = df_pop

# merge jakarta_grouped with neighborhood_df to add latitude/longitude for each neighborhood
denpasar_merged = denpasar_merged.join(denpasar_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

denpasar_merged.head()

,Neighborhood,Population in 2020,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Denpasar Selatan,311590,-8.706134,115.225676,0,Recording Studio,Indonesian Restaurant,Gym / Fitness Center,American Restaurant,Kitchen Supply Store
1,Denpasar Timur,162220,-8.632914,115.246211,1,Bakery,Indian Restaurant,Recording Studio,Food Court,Arcade
2,Denpasar Barat,278020,-8.662244,115.198067,0,Food Truck,Convenience Store,Arcade,Kitchen Supply Store,Asian Restaurant
3,Denpasar Utara,211070,-8.624513,115.209009,0,American Restaurant,Noodle House,Convenience Store,Food Court,Arcade


In [143]:
denpasar_merged['Cluster Labels'].isna().sum()

0

Visualize the cluster

In [144]:
# create map
denpasar_map_clusters = folium.Map(location=[denpasar_latitude, denpasar_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(denpasar_merged['Latitude'], denpasar_merged['Longitude'], denpasar_merged['Neighborhood'], denpasar_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=1).add_to(denpasar_map_clusters)
       
denpasar_map_clusters

In [161]:
den_cluster1=denpasar_merged.loc[denpasar_merged['Cluster Labels'] == 0, denpasar_merged.columns[[0] + list(range(5, denpasar_merged.shape[1]))]]
den_cluster1
den_c1=pd.merge(df_pop,den_cluster1,how='right',on='Neighborhood')
den_c1

,Neighborhood,Population in 2020,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Denpasar Selatan,311590,-8.706134,115.225676,Recording Studio,Indonesian Restaurant,Gym / Fitness Center,American Restaurant,Kitchen Supply Store
1,Denpasar Barat,278020,-8.662244,115.198067,Food Truck,Convenience Store,Arcade,Kitchen Supply Store,Asian Restaurant
2,Denpasar Utara,211070,-8.624513,115.209009,American Restaurant,Noodle House,Convenience Store,Food Court,Arcade


In [163]:
den_cluster2 = denpasar_merged.loc[denpasar_merged['Cluster Labels'] == 1, denpasar_merged.columns[[0] + list(range(5, denpasar_merged.shape[1]))]]
den_cluster2
den_c2=pd.merge(df_pop,den_cluster2,how='right',on='Neighborhood')
den_c2

,Neighborhood,Population in 2020,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Denpasar Timur,162220,-8.632914,115.246211,Bakery,Indian Restaurant,Recording Studio,Food Court,Arcade


### <b> RESULT :  </b>

as we can see the result of K-Mean clustering, first cluster have more dense venues such as recording studio,convenience store, etc and have most populated neighborhood (Denpasar Selatan).The optimum location for new restaurant is :

In [167]:
# Find the geographic center of the most dense or like cluster.
Cluster_0_coorid = den_c1[['Latitude', 'Longitude']]
Cluster_0_coorid = list(Cluster_0_coorid.values) 
lat = []
long = []



for l in Cluster_0_coorid:
  lat.append(l[0])
  long.append(l[1])



Blatitude = sum(lat)/len(lat)
Blongitude = sum(long)/len(long)
print(Blatitude)
print(Blongitude)

-8.66429705
115.21091728122343


In [169]:
# create map
denpasar_map_clusters = folium.Map(location=[denpasar_latitude, denpasar_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(denpasar_merged['Latitude'], denpasar_merged['Longitude'], denpasar_merged['Neighborhood'], denpasar_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=1).add_to(denpasar_map_clusters)
folium.CircleMarker([Blatitude, Blongitude],
                    radius=50,
                    popup='Best',
                    color='red',
                    ).add_to(denpasar_map_clusters)       
denpasar_map_clusters

The best Location to open Fast Food Restaurant is near 500 m at lat : <b> -8.66429705 long 115.21091728122343 or imam bonjol street Denpasar Barat </b>

### <b> DISCUSSION :  </b>

From the resulting cluster in Denpasar we can see that The optimun places to open fast food restaurant is in first cluster, and more detailed in Denpasar Barat Neighborhood based on venue density, and according to population data Denpasar Barat is one of the most populated neighborhood in Denpasar, although in Denpasar Barat there is one fast food restaurant. 

### <b> CONCLUSION :  </b>

As we built our list of neighborhoods with  venues exclusively we discovered most neighborhoods were similar and the greatest concentration of venues was in Denpasar Barat. This might seem obvious but it would also appear that these are some of the most affluent neighborhoods in Denpasar so there appears to be correlation.
I feel confident with this analysis as it is backed up with demonstrated data analysis. While nothing can ever be 100% certain it will helped any business owner to open new Fast Food Restaurant in Denpasar.

